Lösung Aufgabe 2

 In Ihrer Fabrik werden auf fünf verschiedenen Maschinen Schrauben produziert. Die Datei DatAn_PKA25_2.csv
 enthält die gemessenen Längen aller produzierten Schrauben, wobei die Spalte der Nummer der produzierenden Maschine entspricht.
 Untersuchen Sie die Messwerte und beantworten Sie insbesondere die folgenden Fragen:

A. Vorverarbeitung 

In [ ]:
import pandas as pd
import numpy as np

path = "DatAn_PKA25_2.csv"

# Datei ist whitespace-getrennt, keine Header-Zeile
df = pd.read_csv(path, sep=r"\s+", header=None, engine="python")
df.columns = [f"Maschine_{i}" for i in range(1, 6)]

print("Shape:", df.shape)
print(df.head())
print(df.tail())


Ergebnis: 
Shape: (25000, 5) zeigt, es wurden 25.000 Schrauben je Maschine produziert.
Die ersten Zeilen liegen um ~49 mm.
Die letzten Zeilen sind riesig (z.B. ~1.1e16).

Durchführung einer Plausibilitätsprüfung, um Fehler festzustellen



In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(df.iloc[:, 0])
plt.xlabel("Messindex")
plt.ylabel("Schraubenlänge")
plt.title("Messwerte über Messindex")
plt.show()

Man sieht, dass ab Index ca. 2000 die Schraubenlänge kontinuierlich ansteigt. Dies deutet auf einem systematischen Fehler hin. Diese Werte sind unrealistisch und müssen deshalb für die nachfolgende Analyse entfernt werden.

Wir suchen die erste Zeile, in der Werte „viel zu groß“ sind. Hier ist es so eindeutig, dass man sehr konservativ sein kann (z.B. > 1000).

In [ ]:
mask_bad = (df.abs() > 1000).any(axis=1)   # irgendeine Maschine hat >1000
first_bad = mask_bad.idxmax() if mask_bad.any() else None
n_bad = int(mask_bad.sum())

print("Erste auffällige Zeile (0-basiert):", first_bad)
print("Anzahl auffälliger Zeilen:", n_bad)
print(df.iloc[first_bad-3:first_bad+3])



erste auffällige Zeile: 20000
auffällige Zeilen: 5000 (Zeilen 20000–24999)

Direkt um den Bruch sieht man:
Zeile 19999: noch Werte im „normalen“ Bereich (z.B. 30–90…)
Zeile 20000: plötzlich 2028, 10160, 28819, …

Das ist nicht „ein paar Ausreißer“, sondern ein kompletter Datenblock mit anderer Bedeutung/Skalierung.
Wenn man alles „roh“ auswertet, wird z.B. der Mittelwert komplett verfälscht, weil der gigantische Block dominiert:

In [ ]:
print("Rohdaten describe():")
print(df.describe().T[["count","mean","std","min","50%","max"]])


Roh-Mittelwerte liegen bei ca. 5.7e14 (absurd für Längenmessungen),
Median bleibt ~49 (weil Median robust ist).

Dies bestätigt, der Mittelwert ist sensitiv gegenüber Ausreißern. Der Median hingegen ist resistenter

Typische Fragestellung: „Produziert meine Maschine noch Schrauben von 50mm Länge…?“ 
→ Das bestätigt den Größenordnungsrahmen: Werte wie 10^3 bis 10^16 sind kein plausibler Messbereich für diese Aufgabe.

In [ ]:
df_block_ok = df.loc[~mask_bad].copy()   # 20000 Zeilen
df_block_bad = df.loc[mask_bad].copy()  # 5000 Zeilen

print(df_block_ok.shape, df_block_bad.shape)
print(df_block_ok.describe().T[["count","mean","std","min","50%","max"]])


Schritt 1: korrupten Datenblock entfernen/isolieren

df_block_ok: (20000, 5)

Mittelwerte ~50.3–50.6, Median ~49.0, Min ~10–11, Max ~181–211

df_block_bad: (5000, 5)

Min ab ~2028 und hoch bis ~1.1e16 (klar anderer Datensatz/Skala)

Entscheidung: Für die Schraubenlängen-Analyse verwenden wir nur df_block_ok. Den bad-block kann man als „defekte/fehlzugeordnete Messungen“ dokumentieren.

Schritt 2: Ausreißer im plausiblen Block behandeln

Ausreißer erkennt man u.a. daran, dass Messwerte außerhalb eines zulässigen Wertebereichs liegen (physikalisch / Messbereich). 

Behandlungsmöglichkeiten: markieren, korrigieren, als NaN ersetzen, entfernen. 

Für Boxplots: Ausreißer sind Punkte außerhalb von [Q1−1.5·IQR, Q3+1.5·IQR]. 

Da wir (noch) keine Toleranzgrenzen aus der Aufgabenstellung bekommen haben, ist die Boxplot-Regel eine gut begründbare, datengetriebene Standardmethode.

In [ ]:
def boxplot_outlier_info(s):
    s = s.dropna()
    q1 = s.quantile(0.25)
    q3 = s.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    out = (s < lower) | (s > upper)
    return pd.Series({
        "Q1": q1, "Q3": q3, "IQR": iqr,
        "Lower": lower, "Upper": upper,
        "Outlier_n": int(out.sum()),
        "Outlier_frac": float(out.mean())
    })

out_table = df_block_ok.apply(boxplot_outlier_info)
print(out_table.T)

plt.figure(figsize=(8, 5))
df_block_ok.boxplot()
plt.ylabel("Schraubenlänge [mm]")
plt.title("Boxplot der Schraubenlängen je Maschine")
plt.show()


Die Boxplot-Grenzen liegen je Maschine ungefähr bei ~27 mm bis ~71 mm.
Ausreißeranteil je Maschine ca. 9.2% bis 9.9%.

Interpretation: Es gibt viele Werte weit weg vom zentralen Bereich um ~49 mm.
Ob das „Ausschuss“, „falsche Messung“ oder „andere Schraubenvariante“ ist, entscheidet die Aufgabenstellung. Deshalb ist es sauber, Ausreißer zunächst zu markieren (nicht einfach still zu löschen). Das entspricht Option 1/3 aus dem Skript (markieren oder als NaN ersetzen).

In [ ]:
df_clean = df_block_ok.copy()

bounds = {}
for col in df_block_ok.columns:
    s = df_block_ok[col]
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
    bounds[col] = (lower, upper)
    df_clean.loc[(s < lower) | (s > upper), col] = np.nan

print("NaNs nach Ausreißer-Markierung:")
print(df_clean.isna().sum())


1. Geben Sie zunächst Mittelwert, Standardabweichung, Skewness und Kurtosis der Messdaten (nach
 den jeweiligen Maschinen getrennt!) an.

In [ ]:
means = df_block_ok.mean()
means_df = means.rename("Mittelwert").to_frame()
means_df


In [ ]:
stds = df_block_ok.std(ddof=1)  # Stichproben-Standardabweichung (pandas default)
stds_df = stds.rename("Standardabweichung").to_frame()
stds_df

In [ ]:
skew = df_block_ok.skew()
skew_df = skew.rename("Skewness").to_frame()
skew_df


In [ ]:
kurt = df_block_ok.kurt()
kurt_df = kurt.rename("Kurtosis").to_frame()
kurt_df


2. Untersuchen Sie die fünf Verteilungsfunktionen auf Symmetrie. Bei welchen der Verteilungsfunktionen liegt eine Normalverteilung vor? Beschreiben Sie die Abweichung der anderen Verteilungsfunktionen von der Normalverteilung.

In [ ]:
import numpy as np


for col in df_block_ok.columns:
    x = np.sort(df_block_ok[col])
    x_sym = x[::-1]

    plt.figure()
    plt.scatter(x, x_sym, s=2)
    plt.plot([x.min(), x.max()], [x.min(), x.max()])
    plt.title(f"Symmetriediagramm Maschine {col}")
    plt.xlabel("x")
    plt.ylabel("gespiegeltes x")
    plt.show()


In [ ]:
print(skew_df)
print(kurt_df)
from scipy.stats import probplot

for col in df_block_ok.columns:
    plt.figure()
    probplot(df_block_ok[col], dist="norm", plot=plt)
    plt.title(f"QQ-Plot {col}")
    plt.show()


Die Symmetrieuntersuchung zeigt, dass keine der fünf Verteilungsfunktionen symmetrisch ist. Die Symmetriediagramme weisen systematische Abweichungen von der Winkelhalbierenden auf, wobei kleinen Messwerten deutlich größere gespiegelte Werte gegenüberstehen. Dies deutet auf eine ausgeprägte Rechtsschiefe aller Verteilungen hin.

Eine Normalverteilung liegt bei keiner der fünf Maschinen vor. Dies wird durch die QQ-Plots bestätigt: Zwar liegen die Punkte im zentralen Bereich teilweise nahe an der Referenzgeraden, jedoch treten insbesondere in den Randbereichen deutliche systematische Abweichungen auf. Im rechten Rand liegen die Punkte deutlich oberhalb der Geraden, was auf einen schweren rechten Tail hinweist.

Die positive Skewness von etwa 1,7 bestätigt die Rechtsschiefe der Verteilungen, während die hohe Kurtosis von etwa 8 auf deutlich schwerere Tails im Vergleich zur Normalverteilung hindeutet. Die Abweichung von der Normalverteilung äußert sich somit in einer asymmetrischen Verteilung mit stark ausgeprägten extrem großen Werten.

 3. Untersuchen Sie, ob sich die von den verschiedenen Maschinen erzeugten Verteilungsfunktionen
 in ihrer Formvoneinander unterscheiden. Haben zwei der Verteilungsfunktionen die gleiche Form?

In [ ]:
plt.figure()

for col in df_block_ok.columns:
    plt.hist(df_block_ok[col], bins=50, density=True, alpha=0.4)

plt.xlabel("Schraubenlänge [mm]")
plt.ylabel("Dichte")
plt.show()


In [ ]:
from scipy.stats import skew, kurtosis

for col in df_block_ok.columns:
    print(col,
          round(skew(df_block_ok[col]), 2),
          round(kurtosis(df_block_ok[col]), 2))
